In [38]:
# !pip install tf-models-official==2.4.0
!pip install -U tensorflow-text

In [39]:
# !pip install tensorflow_hub

In [40]:
import pandas as pd
import json

In [41]:
# from official.nlp import optimization

In [42]:
import numpy as np

In [43]:
from pandas import json_normalize

In [44]:

file_path = "/content/intents.json"

In [45]:
with open(file_path, 'r') as json_file:
    json_data = json.load(json_file)

In [46]:
column_names = ['message', 'intent']

In [47]:
df = pd.DataFrame(columns=column_names)

In [48]:
df

,message,intent


In [49]:
json_data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hello there!',
    'Hi!',
    'Hey!',
    'Good morning!',
    'Good afternoon!',
    'Good evening!',
    'Hi there, chatbot!',
    'Hey, how are you?',
    'Greetings, chatbot!',
    "What's up, bot?",
    'Hi, buddy!',
    'Hello, dear bot!',
    'Howdy, chatbot!',
    'Yo, bot!',
    'Hey, chat friend!',
    'Hi, virtual assistant!',
    'Hello, AI bot!',
    'Hi, there, clever bot!',
    'Greetings and salutations!',
    'Well, hello there!',
    'Hiya, chatbot!',
    'Good to see you, bot!',
    'Hey, robot friend!',
    "How's it going, chatbot?",
    'Hello, smart AI!',
    'Hi there, helpful bot!',
    'Hey, virtual friend!',
    'Hello, chat partner!',
    'How are you doing today?',
    'Hi, interactive assistant!',
    'Greetings, helpful bot!',
    'Hey, AI buddy!',
    'Hi there, digital companion!',
    'Hello, knowledgeable bot!',
    'Good morning, chatbot friend!',
    'Hey, smart bot, how can you assist me?',
    'Hi,

In [50]:
response = {}

In [51]:
for intent in json_data['intents']:
    for pattern in intent['patterns']:
        new_row = {
            'message' : pattern,
            'intent' : intent['tag']
        }
        df = df.append(new_row, ignore_index = True)


<ipython-input-51-3e72b042036c>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index = True)
<ipython-input-51-3e72b042036c>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index = True)
<ipython-input-51-3e72b042036c>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index = True)
<ipython-input-51-3e72b042036c>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index = True)
<ipython-input-51-3e72b042036c>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

In [52]:
df

,message,intent
0,Hello there!,greeting
1,Hi!,greeting
2,Hey!,greeting
3,Good morning!,greeting
4,Good afternoon!,greeting
...,...,...
522,How do I become invisible? Just testing your s...,fallback
523,"Can you predict my future? You know, like a di...",fallback
524,What's the meaning of this dream I had last ni...,fallback
525,Can you tell me about the meaning of life acco...,fallback


In [53]:
for intent in json_data['intents']:
    for respo in intent['responses']:
        new_key = intent['tag']
        new_value = respo

        if new_key in response:
            response[new_key].append(new_value)
        else:
            response[new_key] = [new_value]

In [54]:
with open("responses.json", "w") as fp:
    json.dump(response , fp)

In [55]:
for key in response.keys():
    print(key)

greeting
book_session
session_cancelation
session_reschedule
feedback
service_completion
fallback


In [56]:
new_df = df.sample(frac=1).reset_index(drop=True)

In [57]:
new_df

,message,intent
0,"Hi, there, chatbot pal!",greeting
1,Could you please help me cancel my appointment?,session_cancelation
2,I'd like to cancel my appointment at your hosp...,session_cancelation
3,Can you cancel the appointment I made for [dat...,session_cancelation
4,I'm at a dead end.,fallback
...,...,...
522,"What's up, bot?",greeting
523,"Hiya, chatbot!",greeting
524,Can you cancel my appointment for me?,session_cancelation
525,"No, that's fine",service_completion


In [58]:
new_df.to_csv('data.csv')

In [59]:
new_df.value_counts('intent')

intent
service_completion     87
fallback               84
session_reschedule     78
session_cancelation    77
book_session           76
feedback               75
greeting               50
dtype: int64

In [60]:
new_df['label'] = new_df.intent.map({
'fallback' :              0,
'service_completion':     1,
'greeting' :              2,
'session_reschedule':     3,
'session_cancelation':    4,
'book_session':           5,
'feedback':               6
})

In [61]:
new_df

,message,intent,label
0,"Hi, there, chatbot pal!",greeting,2
1,Could you please help me cancel my appointment?,session_cancelation,4
2,I'd like to cancel my appointment at your hosp...,session_cancelation,4
3,Can you cancel the appointment I made for [dat...,session_cancelation,4
4,I'm at a dead end.,fallback,0
...,...,...,...
522,"What's up, bot?",greeting,2
523,"Hiya, chatbot!",greeting,2
524,Can you cancel my appointment for me?,session_cancelation,4
525,"No, that's fine",service_completion,1


In [62]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text

In [63]:
new_df.to_csv('data.csv')

In [64]:
encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [65]:
test_text = ["Hi, what are you doing", "where are you going rita", "I have become death, destroyer of worlds"]

In [66]:
preprocess_model = hub.KerasLayer(preprocess)

In [67]:
model_inputs = preprocess_model(test_text)

In [68]:
model_inputs

{'input_mask': <tf.Tensor: shape=(3, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [69]:
bert_model = hub.KerasLayer(encoder)

In [70]:
embeddings = bert_model(model_inputs)

In [71]:
embeddings.keys()

dict_keys(['default', 'pooled_output', 'encoder_outputs', 'sequence_output'])

In [72]:
embeddings['pooled_output'][0].numpy()

array([-9.05384779e-01, -4.02457327e-01, -8.75975311e-01,  8.38889956e-01,
        5.54962516e-01, -2.17419505e-01,  9.22129989e-01,  3.37426633e-01,
       -7.28243709e-01, -9.99984086e-01, -4.53189313e-01,  9.33082342e-01,
        9.78469193e-01,  5.75237513e-01,  9.55716908e-01, -8.11026633e-01,
       -4.74093527e-01, -6.31512046e-01,  2.42031083e-01, -5.89276552e-01,
        6.44507706e-01,  9.99955714e-01,  2.08523616e-01,  2.95355767e-01,
        4.16762739e-01,  9.86542523e-01, -8.56312037e-01,  9.53095853e-01,
        9.61090446e-01,  6.44320905e-01, -7.38614738e-01,  1.64391458e-01,
       -9.85232413e-01, -2.66091436e-01, -8.87619078e-01, -9.90807176e-01,
        3.43408346e-01, -7.83737838e-01, -9.39260498e-02,  1.91215184e-02,
       -8.99714231e-01,  2.81459570e-01,  9.99964833e-01, -2.59163976e-01,
        2.77769983e-01, -3.37039739e-01, -1.00000000e+00,  2.37077370e-01,
       -8.93354297e-01,  8.77555728e-01,  8.81496072e-01,  7.51771688e-01,
        1.67181760e-01,  

In [73]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(new_df['message'], new_df['label'], test_size=0.2, stratify=new_df['label'])

In [74]:
X_train.shape

(421,)

In [75]:
X_test

109    Could you assist me in choosing a doctor's app...
194    I'd like to cancel the appointment I had sched...
137                           No, that's the whole thing
59                               Can I give my thoughts?
375                                        No, that's it
                             ...                        
192                        Hello, intelligent assistant!
280         I want to change the time of my appointment.
307                                                  ???
26          Can you help me by canceling my appointment?
129                I want to plan my visit to the doctor
Name: message, Length: 106, dtype: object

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [76]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
net = tf.keras.layers.Dropout(0.1)(net)
net = tf.keras.layers.Dense(7, activation='softmax', name='classifier')(net)
model = tf.keras.Model(inputs = [text_input], outputs = [net])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 preprocessing (KerasLayer)  {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_type_ids': (None,                                           
                              128),                                                               
                              'input_word_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [77]:
loss = 'sparse_categorical_crossentropy'
metrics = ['accuracy']

In [78]:
# epochs = 10
# steps_per_epoch = 476
# num_train_steps = steps_per_epoch * epochs
# num_warmup_steps = int(0.1*num_train_steps)

init_lr = 1e-5
optimizer = tf.keras.optimizers.AdamW(learning_rate=init_lr)
# optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')

In [79]:
model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [80]:
num_epochs = 13
batch_size = 32
total_samples = 476
steps_per_epoch = total_samples // batch_size
model.fit(X_train, y_train, epochs=num_epochs, steps_per_epoch=steps_per_epoch)

Epoch 1/13
14/14 [==============================] - 53s 797ms/step - loss: 1.9289 - accuracy: 0.2494
Epoch 2/13
14/14 [==============================] - 11s 800ms/step - loss: 1.4876 - accuracy: 0.4228
Epoch 3/13
14/14 [==============================] - 11s 808ms/step - loss: 1.1076 - accuracy: 0.6746
Epoch 4/13
14/14 [==============================] - 11s 820ms/step - loss: 0.7630 - accuracy: 0.8432
Epoch 5/13
14/14 [==============================] - 12s 818ms/step - loss: 0.5119 - accuracy: 0.8979
Epoch 6/13
14/14 [==============================] - 12s 827ms/step - loss: 0.3623 - accuracy: 0.9287
Epoch 7/13
14/14 [==============================] - 12s 827ms/step - loss: 0.2531 - accuracy: 0.9667
Epoch 8/13
14/14 [==============================] - 12s 822ms/step - loss: 0.1738 - accuracy: 0.9929
Epoch 9/13
14/14 [==============================] - 11s 816ms/step - loss: 0.1268 - accuracy: 0.9929
Epoch 10/13
14/14 [==============================] - 11s 818ms/step - loss: 0.0808 - accura

In [81]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 2s 250ms/step - loss: 0.0279 - accuracy: 1.0000


[0.027923796325922012, 1.0]

In [82]:
intent = ['fallback','service_completion','greeting','session_reschedule','session_cancelation','book_session','feedback']

In [83]:
def predict_intent(text):
    x = model.predict([text])
    ind = np.argmax(x)
    return intent[ind]

In [84]:
predict_intent("can you schedule an appointment for me")

1/1 [==============================] - 1s 793ms/step


'book_session'

In [85]:
y_predic = model.predict(X_test)

4/4 [==============================] - 2s 337ms/step


In [86]:
y_pred = [np.argmax(x) for x in y_predic]

In [87]:
from sklearn.metrics import classification_report

In [91]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       1.00      1.00      1.00        18
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00        16
           4       1.00      1.00      1.00        15
           5       1.00      1.00      1.00        15
           6       1.00      1.00      1.00        15

    accuracy                           1.00       106
   macro avg       1.00      1.00      1.00       106
weighted avg       1.00      1.00      1.00       106



In [92]:
model.save("7_intent.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
